In [ ]:
import numpy as np
from numpy import sqrt, pi
import math
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import cadquery as cq
from cadquery import exporters
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, 
                                       Faces, Vertices, show)
from jupyter_cadquery import set_defaults, set_sidecar
set_defaults(theme="dark")
set_sidecar("CadQuery", init=True)

from tqdm import tqdm
from time import perf_counter

import os
from pathlib import Path
cwd=os.getcwd()
home = str(Path(os.path.abspath(cwd)).parents[1])

In [ ]:
# FUNCTIONS

In [ ]:
def hexGen(xVect,yVect):
    for i in range(len(yVect)):
        if i==0:
            yNew=np.ones(len(xVect))*yVect[i]
            xNew=xVect
        else:
            yNew=np.concatenate((yNew, np.ones(len(xVect))*yVect[i]),
                                axis=None)
            xNew=np.concatenate((xNew, xVect), axis=None)
    return xNew, yNew

In [ ]:
def hedronGen(quadside):

    a=sqrt(3)*quadside/2 #edge length
    L=quadside/2
    pts = [
        (L,L),
        (-L,L),
        (-L,-L),
        (L,-L)
    ]
    workplane=cq.Workplane('XY').transformed(offset=cq.Vector(0, 0, -L))
    base=workplane.polyline(pts).close().extrude(quadside)
    
    dx=quadside #in fact dy
    dz=quadside #in fact dx
    dy=quadside/2 #in fact dz
    wedge1=workplane.transformed(offset=cq.Vector(0,0,dy/2+quadside),rotate=
                                 cq.Vector(90,0,0)).wedge(dx, dy, dz, dx/2, 
                                 dz/2, dx/2, dz/2, pnt=cq.Vector(0.0, 0.0, 
                                 0.0), dir=cq.Vector(0.0, 0.0, 1.0))
    wedge2=workplane.workplane(invert=True).transformed(offset=cq.Vector(0,0,
                               dy/2),rotate=cq.Vector(90,0,0)).wedge(dx, dy, 
                               dz, dx/2, dz/2, dx/2, dz/2, pnt=cq.Vector(0.0, 
                               0.0, 0.0), dir=cq.Vector(0.0, 0.0, 1.0))
    wedge3=workplane.transformed(offset=cq.Vector(0,3*dy/2,quadside/2),
                                 rotate=cq.Vector(0,90,0)).wedge(dx, dy, dz, 
                                 dx/2, dz/2, dx/2, dz/2, pnt=cq.Vector(0.0, 
                                 0.0, 0.0), dir=cq.Vector(0.0, 0.0, 1.0))
    wedge4=workplane.transformed(offset=cq.Vector(0,-quadside/2-dy/2,quadside
                                 /2),rotate=cq.Vector(0,90,0)).workplane(
                                 invert=True).wedge(dx, dy, dz, dx/2, dz/2, 
                                 dx/2, dz/2, pnt=cq.Vector(0.0, 0.0, 0.0), 
                                 dir=cq.Vector(0.0, 0.0, 1.0))
    wedge5=workplane.transformed(rotate=cq.Vector(0,0,90)).transformed(
                                 offset=cq.Vector(0,quadside/2+dy/2,
                                 quadside/2),rotate=cq.Vector(0,90,0)).wedge(
                                 dx, dy, dz, dx/2, dz/2, dx/2, dz/2, pnt=
                                 cq.Vector(0.0, 0.0, 0.0), dir=cq.Vector(
                                 0.0, 0.0, 1.0))
    wedge6=workplane.transformed(rotate=cq.Vector(0,0,90)).transformed(
                                 offset=cq.Vector(0,dy/2-quadside,
                                 quadside/2),rotate=cq.Vector(0,90,
                                 0)).workplane(invert=True).wedge(dx, dy, dz,
                                 dx/2, dz/2, dx/2, dz/2, pnt=cq.Vector(0.0, 
                                 0.0, 0.0), dir=cq.Vector(0.0, 0.0, 1.0))
    
    rhombicDodecahedron=(base.union(wedge1,glue=True,clean=True).union(
                         wedge2,glue=True,clean=True).union(wedge3,glue=True,
                         clean=True).union(wedge4,glue=True,clean=True
                         ).union(wedge5,glue=True,clean=True).union(wedge6,
                         glue=True, clean=True))
    #rhombicDodecahedron=rhombicDodecahedron.translate(cq.Vector(X,Y,Z))
    
    return rhombicDodecahedron

In [ ]:
def abcGen(L,T,a1,X1,X2,Y1,Y2,Z1,Z2,A1,A2):
    sD=cq.Workplane('XY').transformed(offset=cq.Vector(0, 0, Z1[0])).rect(
        L,T,centered=True,forConstruction=False).extrude(a1)
    
    # the box is ready
    hedra=[]
    
    for i in tqdm(range(X1.size)):
        hedra.append(hedronGen(A1[i]).translate(cq.Vector(X1[i],Y1[i],
                                                          Z1[i])))

    for i in tqdm(range(X2.size)):
        hedra.append(hedronGen(A2[i]).translate(cq.Vector(X2[i],Y2[i],
                                                          Z2[i])))
    
    for i in tqdm(range(len(hedra))):
        sD=sD.cut(hedra[i],clean=True)

    return sD

In [ ]:
def densify(R,VF,d):
    volFrac=np.ones(len(R))
    for i in range(len(R)):
        if R[i]<=(d/2):
            volFrac[i]=1
        else:
            volFrac[i]=np.exp(-0.4*(R[i]-d/2)+np.log(1-VF))+VF
    return volFrac

In [ ]:
def calcSideLen(volFrac,a1):
    A=np.zeros(len(volFrac))
    for i in range(len(volFrac)):
        A[i]=a1*((1-volFrac[i])**(1./3.))
    return A

In [ ]:
def calcDist(X,Y):
    R=np.zeros(len(X))
    for i in range(len(R)):
        R[i]=sqrt(X[i]*X[i]+Y[i]*Y[i])
    return R

In [ ]:
def remPts(X,Y,Z,R,vF,A,d):
    rem_ind=[]
    for i in range(len(R)):
        if (R[i]-A[i])<(d/2) or A[i]<=0.01:
            rem_ind.append(i)
    rem_ind.reverse()
    for i in range(len(rem_ind)):
        X=np.delete(X,rem_ind[i])
        Y=np.delete(Y,rem_ind[i])
        Z=np.delete(Z,rem_ind[i])
        R=np.delete(R,rem_ind[i])
        vF=np.delete(vF,rem_ind[i])
        A=np.delete(A,rem_ind[i])
    return X, Y, Z, R, vF, A

In [ ]:
def speak(text):
    from IPython.display import Javascript as js, clear_output
    # Escape single quotes
    text = text.replace("'", r"\'")
    display(js(f'''
    if(window.speechSynthesis) {{
        var synth = window.speechSynthesis;
        synth.speak(new window.SpeechSynthesisUtterance('{text}'));
    }}
    '''))
    # Clear the JS so that the notebook doesn't speak again when reopened
    clear_output(False)

In [ ]:
# DATA

VF=0.27         #Volume Fraction
minCellSize=0.5 #[mm] Minimum Cell Size
maxCellSize=1.0 #[mm] Maximum Cell Size
L=10            #[mm] Min Length (x) of Sample
T=10            #[mm] Min Width (y) of Sample
U=12            #[mm] Min Height (z) of Sample
holeDiam=2.7    #[mm] Diameter of Drilled Hole
dImpl=3.8       #[mm] Greatest Outer Diameter of the Implant

isotropicTestBodyGen=0 #1 for isotropic test body, 0 for regular run
numberOfLayers=5       #control test body size, no need to change

#based on Min and Max Cell Size, sphere equivalent
a0=((minCellSize+maxCellSize)/4)*((2*math.pi/3)**(1./3.))
a1=a0/((1-VF)**(1./3.))

if isotropicTestBodyGen: #turn on for isotropic test body generation

    L=numberOfLayers*2*a1
    T=L
    U=L
else:
    L=math.ceil(L/(2*a1))*2*a1
    T=math.ceil(T/(2*a1))*2*a1
    U=math.ceil(U/(2*a1))*2*a1

if isotropicTestBodyGen:
    fileNameSpecifyer=("_VF"+str(VF)+"_CS"+"{:.1f}".format(minCellSize)+"-"
    +"{:.1f}".format(maxCellSize)+"_NoL"+str(numberOfLayers)+"_LTU"
    +"{:.5f}".format(L)+"_ISO")
else:
    fileNameSpecifyer=("_VF"+str(VF)+"_CS"+"{:.1f}".format(minCellSize)+"-"
    +"{:.1f}".format(maxCellSize)+"_L"+"{:.4f}".format(L)+"_T"
    +"{:.4f}".format(T)+"_U"+"{:.4f}".format(U))

print(fileNameSpecifyer)

D=2*a1 #Distance between Centres on the x axis
H=a1 #Distance between Centres on the y axis


Nxa=math.ceil(L/D+1)
if Nxa%2 != 1: #if not odd
    Nxb=Nxa+2 #then with +2 it is good for b-row
    Nxa=Nxa+1 #and has to be incremented for a-row
else: #if odd
    Nxb=Nxa+1 #then a is good, b must be incremented
    
Ny=math.ceil(T/H+1)
if Ny%2 != 1: #if not odd
    Ny=Ny+1 #then increment it
if ((Ny-1)/2)%2 != 1: #if not odd
    Nya=int((Ny+1)/2)
    Nyb=int(Nya+1)
else:
    Nyb=int((Ny+1)/2)
    Nya=int(Nyb+1)
Ny=Nya+Nyb
print("a0 = ",a0,"\nL = ",L,"\nT = ",T,"\nU = ",U,"\nD = ", D, 
      "\nH = ", H, "\nNx:\n\tNxa = ", Nxa, "\n\tNxb = ", Nxb, "\nNy = ", Ny,
      "\n\tNya = ", Nya, "\n\tNyb = ", Nyb)

In [ ]:
# LAYER 'A'

aX=np.linspace(-((Nxa-1)*D)/2, ((Nxa-1)*D)/2, num=Nxa)
bX=np.linspace(-((Nxa-1)*D)/2-(D/2), ((Nxa-1)*D)/2+(D/2), num=Nxb)

if Nya>Nyb:
    aY=np.linspace(-((Nya+Nyb-1)*H)/2, ((Nya+Nyb-1)*H)/2, num=Nya)
    bY=np.linspace(-((Nya+Nyb-1)*H)/2+H, ((Nya+Nyb-1)*H)/2-H, num=Nyb)
else:
    bY=np.linspace(-((Nya+Nyb-1)*H)/2, ((Nya+Nyb-1)*H)/2, num=Nyb)
    aY=np.linspace(-((Nya+Nyb-1)*H)/2+H, ((Nya+Nyb-1)*H)/2-H, num=Nya)

aX0=aX
aY0=aY
bX0=bX
bY0=bY
AaX, AaY = hexGen(aX0,aY0)
AbX, AbY = hexGen(bX0,bY0)

#%matplotlib widget
#plt.plot(AaX,AaY,'b.')
#plt.plot(AbX,AbY,'r.')
#plt.axis('equal')
#plt.show()

In [ ]:
# LAYER 'B'

BaX=AaX
BaY=AaY-a1
BbX=AbX
BbY=AbY-a1

if Nya>Nyb: #'a' row is on the outside
    BbX=np.concatenate((BbX, bX0), axis=None)
    BbY=np.concatenate((BbY, np.ones(len(bX0))*(BaY[-1]+H)), axis=None)
        
else: #'b' row is on the outside
    BaX=np.concatenate((BaX, aX0), axis=None)
    BaY=np.concatenate((BaY, np.ones(len(aX0))*(BbY[-1]+H)), axis=None)

#%matplotlib widget
#plt.plot(AaX,AaY,'b.')
#plt.plot(AbX,AbY,'r.')
#plt.plot(BaX,BaY,'g.')
#plt.plot(BbX,BbY,'y.')
#plt.axis('equal')
#plt.show()

In [ ]:
AX=np.concatenate((AaX, AbX), axis=None)
AY=np.concatenate((AaY, AbY), axis=None)
BX=np.concatenate((BaX, BbX), axis=None)
BY=np.concatenate((BaY, BbY), axis=None)

#%matplotlib widget
#plt.plot(AX,AY,'g.')
#plt.plot(BX,BY,'b.')
#plt.axis('equal')
#plt.show()

In [ ]:
AZ=np.zeros((len(AX)))
BZ=np.ones((len(BX)))*a1

X=np.concatenate((AX,BX), axis=None)
Y=np.concatenate((AY,BY), axis=None)

R=calcDist(X,Y)

if not isotropicTestBodyGen:
    zeroIndex=np.where(R==0)[0][0]
    X=np.delete(X,zeroIndex,None)
    Y=np.delete(Y,zeroIndex,None)
    R=np.delete(R,zeroIndex,None)

In [ ]:
if not isotropicTestBodyGen:
    volFrac=densify(R,VF,dImpl)
else:
    volFrac=np.ones(len(R))*VF #turn on for isotropic test body generation

A=calcSideLen(volFrac,a1)

In [ ]:
fig = px.scatter(x=X, y=Y, color=volFrac, labels={"color":"Volume Fraction"},
                 title="Volume Fraction",width=700,height=700,
                 color_continuous_scale='Plasma')
fig.show()
#fig.write_html(home+"\\img\\python\\generateStructure_final_fig_03.html")
#fig.write_image(home+"\\img\\python\\generateStructure_final_fig_03.svg")

In [ ]:
fig = px.scatter(x=X, y=Y, color=A, labels={"color":"Side Length [mm]"},
                 title="Side Length",width=700,height=700)
fig.show()
#fig.write_html(home+"\\img\\python\\generateStructure_final_fig_04.html")
#fig.write_image(home+"\\img\\python\\generateStructure_final_fig_04.svg")

In [ ]:
RA=calcDist(AX,AY)
RB=calcDist(BX,BY)

if not isotropicTestBodyGen:
    zeroIndex=np.where(RA==0)[0][0]
    AX=np.delete(AX,zeroIndex,None)
    AY=np.delete(AY,zeroIndex,None)
    AZ=np.delete(AZ,zeroIndex,None)
    RA=np.delete(RA,zeroIndex,None)

    volFracA=densify(RA,VF,dImpl)
    volFracB=densify(RB,VF,dImpl)

    AA=calcSideLen(volFracA,a1)
    AB=calcSideLen(volFracB,a1)

    AX, AY, AZ, RA, volFracA, AA = remPts(AX, AY, AZ, RA, 
                                          volFracA, AA, dImpl)
    BX, BY, BZ, RB, volFracB, AB = remPts(BX, BY, BZ, RB, 
                                          volFracB, AB, dImpl)
    
else:
    volFracA=np.ones(len(RA))*VF
    volFracB=np.ones(len(RB))*VF
    
    AA=calcSideLen(volFracA,a1)
    AB=calcSideLen(volFracB,a1)
    
    
# AZ-> AZ1, AZ2
AZ1=AZ
AZ2=AZ+2*a1

In [ ]:
print(len(AX),len(BX))

In [ ]:
sDA=abcGen(L,T,a1,AX,BX,AY,BY,AZ1,BZ,AA,AB)
show(sDA, grid=True)

In [ ]:
if not isotropicTestBodyGen:
    tic = perf_counter()
    exporters.export(sDA, home+'\\out\\step\\BAM_A'+fileNameSpecifyer
                     +'.step')
    tac = perf_counter()
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
sDB=abcGen(L,T,a1,BX,AX,BY,AY,BZ,AZ2,AB,AA)
show(sDB, grid=True)

In [ ]:
if not isotropicTestBodyGen:
    tic = perf_counter()
    exporters.export(sDB, home+'\\out\\step\\BAM_B'+fileNameSpecifyer
                     +'.step')
    tac = perf_counter()
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
tic = perf_counter()
sDAB=sDA.union(sDB,glue=True)
show(sDAB,grid=True)
tac = perf_counter()
print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
if not isotropicTestBodyGen:
    tic = perf_counter()
    exporters.export(sDAB, home+'\\out\\step\\BAM_AB'+fileNameSpecifyer
                     +'.step')
    tac = perf_counter()
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
Zsteps=math.ceil(math.log((U/(a1)), 2))
Zmax=pow(2,Zsteps-1)*2*a1
diffZ=Zmax-U

In [ ]:
tic = perf_counter()
for i in tqdm(range(Zsteps)):
    if i==0:
        sDGlued=sDAB
    else:
        sDGlued=sDGlued.union(sDGlued.translate(cq.Vector(0,0,pow(2,i-1)
                                                          *2*a1)),glue=True)
show(sDGlued,grid=True)
tac = perf_counter()
speak("Glued all layers.")
print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
if not isotropicTestBodyGen:
    tic = perf_counter()
    exporters.export(sDGlued, home+'\\out\\step\\BAM_sDGlued'
                     +fileNameSpecifyer+'.step')
    tac = perf_counter()
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
tic = perf_counter()
slicer=cq.Workplane('XY').transformed(offset=cq.Vector(0,0,U)).rect(L+1,T+1,
                                      centered=True,forConstruction=
                                      False).extrude(diffZ+1)
sDFinal=sDGlued.cut(slicer,clean=True)
show(sDFinal)
tac = perf_counter()
speak("Cut model to size.")
print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
tic = perf_counter()
exporters.export(sDFinal, home+'\\out\\step\\BAM_sDFinal'+fileNameSpecifyer
                 +'.step')
tac = perf_counter()
print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
if not isotropicTestBodyGen:    
    tic = perf_counter()
    hole=cq.Workplane('XY').circle(holeDiam/2).extrude(U)
    sDFinal=sDFinal.cut(hole,clean=True)
    show(sDFinal)
    tac = perf_counter()
    speak("Drilled the hole.")
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
if not isotropicTestBodyGen:
    tic = perf_counter()
    exporters.export(sDFinal, home+'\\out\\step\\BAM_sDFinal'+fileNameSpecifyer
                     +'.step')
    tac = perf_counter()
    print("Elapsed time:", '%.5f' % (tac-tic) , "s")

In [ ]:
speak("All done.")